# Gibbs State Preparation

### Gibbs state preparation using Variational Quantum Imaginary Time Evolution (VarQITE)

The Gradient Module facilitates some sophisticated algorithms such as VarQITE for Gibbs state preparation.

A Gibbs state is defined by a (possibly parameterized) Hamiltonian $H\left(\theta\right)$ and a temperature $T$
$$
\rho^{\text{Gibbs}} = \frac{e^{-H_{\theta}/\left(\text{k}_{\text{B}}\text{T}\right)}}{Z}
$$
with $k_B$ denoting the Botltzmann constant and $Z=\text{Tr}\left[e^{-H_{\theta}/\left(\text{k}_{\text{B}}\text{T}\right)}\right]$.

For further information on Gibbs state preparation using VarQITE, we would like to refer you to the following work [Variational Quantum Boltzmann Machines](https://arxiv.org/abs/2006.06004).

The core of this Gibbs state preparation routine is a parameter propagation that uses a `NaturalGradient`.

In [ ]:
import numpy as np
import scipy as sp

from qiskit.quantum_info import partial_trace

from qiskit.aqua.operators import Z, I, X, StateFn, CircuitStateFn, SummedOp, PauliOp
from qiskit.aqua.operators.evolutions.varqtes.varqite import VarQITE

from qiskit.circuit.library import EfficientSU2, RealAmplitudes
from qiskit.circuit import Parameter

The initial state has to be the maximally mixed state. Thus, we add an additional register of working qubits such that
$$\rho_{init} = \frac{1}{2^n}I = \text{Tr}_B\left[\left(\bigotimes\limits_{i=0}^{n-1}|00\rangle+|11\rangle\right)\left(\bigotimes\limits_{i=0}^{n-1}\langle00|+\langle11|\right) \right].$$

Setting our Hamiltonian and extending it with a trivial part (the identity) on the working qubits enables us to use VarQITE for pure states to conduct the Gibbs State preparation.

More explicitly, we evolve the parameters of a chosen Ansatz according to VarQITE for pure states for $2n$ qubits and finally trace out the working qubits to get the wanted approximation to our Gibbs state.

In [55]:
# Can't use this right away there is a problem in the decomposition for the gradient_framework
H_target = SummedOp([Z ^ Z, I ^ X])
H = (I ^ H_target.num_qubits) ^ H_target

Now, we choose a temperature $T$.

In [56]:
kbT = 1

Available Ansatz Choices

In [57]:
# Instantiate the model ansatz
depth = 2

# Efficient SU2 - based
# entangler_map = [[i+1, i] for i in range(H.num_qubits - 1)]
ansatz = EfficientSU2(H.num_qubits, reps=depth, entanglement = 'sca')
qr = ansatz.qregs[0]
for i in range(int(H.num_qubits/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Real Amplitudes - based
# ansatz = RealAmplitudes(H.num_qubits, reps=depth, entanglement = 'sca')
# qr = ansatz.qregs[0]
# for i in range(int(H.num_qubits/2)):
#     ansatz.cx(qr[i], qr[i+int(H.num_qubits/2)])


In [58]:
ansatz

In [59]:
# Initialize the Ansatz parameters

# For Efficient SU2 - based
param_values_init = np.zeros(2 * H.num_qubits * (depth + 1))
for j in range(2 * H.num_qubits * depth, int(len(param_values_init) - H.num_qubits - 2)):
    param_values_init[int(j)] = np.pi/2.
    
# # For Real Amplitudes - based
# param_values_init = np.zeros(H.num_qubits * (depth + 1))
# for j in range(H.num_qubits * depth, int(len(param_values_init) - H.num_qubits / 2.)):
#     param_values_init[j] = np.pi / 2.
    

In [60]:
param_values_init

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.57079633, 1.57079633, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [61]:
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz    
op = ~StateFn(H) @ CircuitStateFn(ansatz)
# Now, we set the evolution time according to the chosen effective temperature kBT
op = 1/(2*kbT) * op

# Define the discretization grid of the time steps
num_time_steps = 10


In [62]:
gibbs_purified = VarQITE(parameters=ansatz.ordered_parameters, get_error=True, grad_method='lin_comb', 
                         qfi_method = 'lin_comb_full', init_parameter_values=param_values_init, 
                         num_time_steps=num_time_steps,
                         fidelity_to_target=True, regularization='ridge').convert(op)

Grad error (1.224827+0j)
Error (5.421+0j) after time step 0
Fidelity 0.9987190392798984
True error 0.18919895768000555
Grad error (1.225071+0j)
Error (8.884+0j) after time step 1
Fidelity 0.9950530031333815
True error 0.26528943114811354
Grad error (1.225791+0j)
Error (11.097+0j) after time step 2
Fidelity 0.9894666679806314
True error 0.32057423746418184
Grad error (1.226943+0j)
Error (12.512+0j) after time step 3
Fidelity 0.9825965399244935
True error 0.3636092735555199
Grad error (1.228459+0j)
Error (13.417+0j) after time step 4
Fidelity 0.9751201673886309
True error 0.3977819537592658
Grad error (1.230247+0j)
Error (13.996+0j) after time step 5
Fidelity 0.9676449101600034
True error 0.424989119316787
Grad error (1.232208+0j)
Error (14.366+0j) after time step 6
Fidelity 0.9606398880547803
True error 0.44653313911906684
Grad error (1.234243+0j)
Error (14.603+0j) after time step 7
Fidelity 0.9544134445762661
True error 0.4634208183156654
Grad error (1.236256+0j)
Error (14.755+0j) afte

In [63]:
gibbs_purified_res = gibbs_purified.eval().primitive

In [64]:
gibbs_purified_array = gibbs_purified_res.data

In [65]:
gibbs_target = sp.linalg.expm((-1)*H_target.to_matrix()/kbT)
gibbs_target /= np.trace(gibbs_target)
print(np.round(gibbs_target, 3))

[[ 0.093+0.j -0.157+0.j  0.   +0.j  0.   +0.j]
 [-0.157+0.j  0.407+0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.407+0.j -0.157+0.j]
 [ 0.   +0.j  0.   +0.j -0.157+0.j  0.093+0.j]]


In [66]:
gibbs = partial_trace(gibbs_purified_array, [2, 3])

In [67]:
np.round(gibbs.data, 3)

array([[ 0.142+0.j, -0.1  +0.j,  0.   +0.j,  0.   -0.j],
       [-0.1  -0.j,  0.358+0.j,  0.   -0.j,  0.001-0.j],
       [ 0.   -0.j,  0.   +0.j,  0.358+0.j, -0.1  +0.j],
       [ 0.   +0.j,  0.001+0.j, -0.1  -0.j,  0.143+0.j]])

# QBM

In [73]:
from qiskit.algorithms.optimizers import L_BFGS_B

a = Parameter('a')
b = Parameter('b')
c = Parameter('c')
H_params = [a, b, c]
H_target = SummedOp([a * Z ^ Z, b * I ^ Z, c * Z ^ I])
H = (I ^ H_target.num_qubits) ^ H_target

# Define the discretization grid of the Gibbs state prep
num_time_steps = 10

# Choose initial values for Hamiltonian parameters
p_H_values_init = np.random.rand(3) * 2 - 2

# Set the optimizer
optimizer = L_BFGS_B(maxiter=100, iprint=5)

target_prob = [0.5, 0., 0., 0.5]

# Define the loss function

def loss(params):
    p_dict = dict(zip(H_params, params))
    hamiltonian = H.assign_parameters(p_dict)

    # Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz    
    op = ~StateFn(hamiltonian) @ CircuitStateFn(ansatz)
    # Now, we set the evolution time according to the chosen effective temperature kBT
    op = 1/(2*kbT) * op
    # Prepare the Gibbs State
    
    """
    @Jul - Replace the VarQITE Gibbs Prep with your Nat Grad SPSA 
    Taking op and 'propagating' the parameters as p + dt * nat_grad_spsa_res
    """ 
    gibbs_purified = VarQITE(parameters=ansatz.ordered_parameters, get_error=True, grad_method='lin_comb', 
                         qfi_method = 'lin_comb_full', init_parameter_values=param_values_init, 
                         num_time_steps=num_time_steps,
                         fidelity_to_target=True, regularization='ridge').convert(op)
    
    gibbs_purified_res = gibbs_purified.eval().primitive
    gibbs_purified_array = gibbs_purified_res.data
    gibbs_prob = np.diag(partial_trace(gibbs_purified_array, [2, 3]).data)
    
    gibbs_target = sp.linalg.expm((-1)*H_target.to_matrix()/kbT)
    gibbs_target /= np.trace(gibbs_target)
    gibbs_target_prob = np.diag(gibbs_target)

    return (-1) * np.dot(target_prob, np.log(gibbs_prob))

res = optimizer.optimize(3, loss, initial_point = p_H_values_init)


CircuitError: 'Bound parameter expression is complex in gate u1'